In [2]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
import redshift_connector
import botocore 
import botocore.session 
from aws_secretsmanager_caching import SecretCache, SecretCacheConfig 

In [10]:
# Getting Redshift Password
client = botocore.session.get_session().create_client('secretsmanager')
cache_config = SecretCacheConfig()
cache = SecretCache( config = cache_config, client = client)

RS_PASSWORD = cache.get_secret_string('redshift_db_password')

# CONFIGURATION
RS_HOST = "ds-workgroup.425768274696.eu-west-2.redshift-serverless.amazonaws.com"
RS_DATABASE = "ds_redshift_db"
RS_USER = "postgres"
RS_SCHEMA = "public"
RS_TABLE = "synthetic_data"
REGION = "eu-west-1"
S3_BUCKET  = 'juan-test-redshift'
S3_OBJECT  = 'test2.gz.jsonl'
RS_IAM_ARN = "arn:aws:iam::425768274696:role/redshift_role"


conn = redshift_connector.connect(
    host=RS_HOST,
    database=RS_DATABASE,
    user=RS_USER,
    password=RS_PASSWORD,
    ssl=False
)
cursor = conn.cursor()

In [5]:
print(RS_PASSWORD)

[YWOwGCai2]062nb


In [12]:
cursor.execute("set statement_timeout = 360000")

cursor.execute(f"CREATE TABLE IF NOT EXISTS {RS_TABLE} (commons SUPER);")

cursor.execute(f"COPY public.{RS_TABLE} from 's3://{S3_BUCKET}/{S3_OBJECT}' iam_role '{RS_IAM_ARN}' json 'noshred' GZIP REGION '{REGION}';")

conn.commit()
cursor.close()
conn.close()